In [2]:
import os

In [3]:
%pwd

'e:\\Projects-GenAI\\Medical_Chatbot_Llama2\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'e:\\Projects-GenAI\\Medical_Chatbot_Llama2'

In [6]:

os.environ['PINECONE_API_KEY']='c46f8c74-f32b-4b43-ba41-c6f8808124b3'

In [7]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = 'chatbot'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="chatbot",
        dimension=384, 
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 


e:\Projects-GenAI\Medical_Chatbot_Llama2\env8\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:

from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
import time

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf('data')

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

text_chunks = text_split(extracted_data)

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    return embeddings

embeddings = download_huggingface_embeddings()

# Embed each chunk and upsert the embeddings into your Pinecone index.
# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name="chatbot",
#     embedding=embeddings, 
#     namespace="default" 
# )

time.sleep(1)


e:\Projects-GenAI\Medical_Chatbot_Llama2\env8\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
e:\Projects-GenAI\Medical_Chatbot_Llama2\env8\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environ

In [9]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index = pc.Index("medical-chatbot")
namespace = "default"

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0], 
        namespace=namespace, 
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '00091a24-5dab-4e3a-b344-c29a9828cd4c',
              'metadata': {'page': 391.0,
                           'source': 'data\\Medical_book.pdf',
                           'text': 'lates. Other members of this group include '
                                   'sodium salicy-late, choline salicylate, '
                                   'and magnesium salicylate. Thesedrugs are '
                                   'more expensive and no more effective '
                                   'thanaspirin. However, they are a little '
                                   'easier on the stomach.Aspirin is quickly '
                                   'absorbed into the bloodstream and '
                                   'pro-vides quick and relatively '
                                   'long-lasting pain relief. Aspirinalso '
                                   'reduces inflammation. Researchers believe '
                                   'theseeffects come about because 

In [9]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [10]:
from langchain.prompts import PromptTemplate
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [10]:
from langchain.chains import RetrievalQA 
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
import os

# Initialize a LangChain object for chatting with the LLM
# without knowledge from Pinecone.
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

# Initialize a LangChain object for retrieving information from Pinecone.
knowledge = PineconeVectorStore.from_existing_index(
    index_name="medical-chatbot",
    namespace="default",
    embedding=embeddings
)

# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone. 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=knowledge.as_retriever()
)

In [40]:
user_input='what is acne?'
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :   Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [24]:
# from pinecone_text.sparse import BM25Encoder

# bm25_encoder = BM25Encoder().default()


In [25]:
# bm25_encoder.fit([t.page_content for t in text_chunks])

# bm25_encoder.dump('bm25_values.json')

# bm25_encoder = BM25Encoder().load('bm25_values.json')

100%|██████████| 7020/7020 [00:21<00:00, 324.16it/s]


In [26]:
# index = pc.Index("chatbot")

In [27]:
# from langchain.retrievers.pinecone_hybrid_search import PineconeHybridSearchRetriever
# retrieval = PineconeHybridSearchRetriever(embeddings=embeddings, 
#                                           index=index, 
#                                           sparse_encoder=bm25_encoder)

In [28]:
# retrieval

PineconeHybridSearchRetriever(tags=None, metadata=None, embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001EA81E47D90>, index=<pinecone.grpc.index_grpc.GRPCIndex object at 0x000001EA7DF29D00>, top_k=4, alpha=0.5)

In [29]:
# retrieval.add_texts([t.page_content for t in text_chunks])

100%|██████████| 220/220 [32:36<00:00,  8.89s/it] 


In [ ]:
# retrieval.invoke('what is acne')